In [ ]:
from google.colab import files
from google.colab import drive

uploaded = files.upload()

drive.mount('/content/drive')

Saving link_list3.csv to link_list3.csv
Mounted at /content/drive


In [ ]:
! rm -rf sample_data

In [ ]:
import pandas as pd
import os
import io

from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import time # Calculate duration of code execution

pd.set_option("display.max_colwidth", None)

In [ ]:
def get_start_time():
    return time.time()

def end_time(start_time):
    # Get the end time
    end_time = time.time()

    # Calculate the duration in seconds
    duration = end_time - start_time

    # Convert duration to a more readable format
    minutes = duration // 60
    seconds = duration % 60

    # Print the duration
    print(f"Code execution time: {minutes:.0f} minutes {seconds:.2f} seconds")


In [ ]:
raw_data = pd.read_csv(io.BytesIO(uploaded['link_list3.csv']))


# Select only the first three rows
truncated_data = raw_data.head(1000)

# Rename the DataFrame to test_data
test_data = truncated_data.copy()

# Print the test_data
test_data

,link
0,https://www.carlist.my/used-cars/ori-2018-mercedes-benz-cla200-1-6-night-edition-sport-turbo-good-condition/11069776
1,https://www.carlist.my/used-cars/ori-2019-nissan-serena-2-0-s-hybrid-high-way-star-mpv-1-owner-good-condition-warranty/12303748
2,https://www.carlist.my/used-cars/toyota-corolla-altis-1-8-e-max-loan-get-full-body-kit-1-yr-warranty/12089867
3,https://www.carlist.my/used-cars/good-price-good-condition-with-low-mileage-white-colour-detail-honda-br-v-1-5-v-i-vtec-suv/12269039
4,https://www.carlist.my/used-cars/2019-perodua-myvi-1-5-av-best-hatchback-principal-warranty/12217324
...,...
995,https://www.carlist.my/used-cars/2013-proton-preve-1-6-executive-sedan-a-r3-free-1-year-warranty/12299687
996,https://www.carlist.my/used-cars/mercedes-glc300-2-0-amg-line-4matic-amg-bodykits-amg-sport-rim-360-reverse-cam-panoramic-roof-red-black-leather-seat-lka-bsm-precrash-trapomat/12055746
997,https://www.carlist.my/used-cars/2016-mercedes-benz-a250-2-0-sport-amg-full-stock-baru-ori-t-top-cdt-warranty-foru/12246891
998,https://www.carlist.my/used-cars/2010-porsche-panamera-3-6-4-hatchback-a-free-3-year-warranty/11802859


In [ ]:
# Get the start time
start_time = get_start_time()

# Initialize an empty list to store the scraped data
data_list = []

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    }

for index, url in enumerate(test_data['link']):
    print("Scraping link {}...".format(index))

    all_data = requests.get(url, headers=HEADERS).text

    soup = BeautifulSoup(all_data, "html.parser")

    # Extracting individual elements from the page
    cost_element = soup.find('div', class_='listing__item-price')
    if cost_element:
        cost_elements = cost_element.text.strip()
        # Extracting the monthly payment and price from cost_elements
        cost_values = cost_elements.split('\n')
        monthly_payment = cost_values[0].strip()
        price = cost_values[1].strip() if len(cost_values) >= 2 else ''
    else:
        monthly_payment = ''
        price = ''

    # Initialize a dictionary to store the scraped data for each link
    data_dict = {
        'link': url,
        'monthly_payment': monthly_payment,
        'price': price
    }

    # Get the manufacturing year from the 'element' list if it exists
    year_element = soup.find_all('span', class_='u-text-bold u-block')
    if len(year_element) >= 2:
        year = year_element[1].text.strip()
        data_dict['manufacturing_date'] = year
    else:
        data_dict['manufacturing_date'] = ''

    # Get the mileage from the 'element' list if it exists
    mileage_element = soup.find_all('span', class_='u-text-bold u-block')
    if len(mileage_element) >= 2:
        mileage = mileage_element[2].text.strip()
        data_dict['mileage'] = mileage
    else:
        data_dict['mileage'] = ''

    # Extracting individual elements from the specification page
    specification_elements = soup.find_all('div', class_='u-width-1/2 u-width-1@mobile u-padding-sides-md')

    # Storing the data in the same dictionary for specification_elements
    for element in specification_elements:
        left_span = element.find('span', class_='u-width-1/2')
        right_span = element.find('span', class_='u-text-bold u-width-1/2 u-align-right')
        if left_span and right_span:
            left_span_text = left_span.text.strip()
            right_span_text = right_span.text.strip()
            data_dict[left_span_text] = right_span_text

    # Append the dictionary to the list
    data_list.append(data_dict)

    # Add a delay of 3 seconds between requests
    time.sleep(3)

# Convert the list of dictionaries into a pandas DataFrame
df_result = pd.DataFrame(data_list)

# Call the end_time function and pass the start time as an argument
end_time(start_time)

from google.colab import files
df_result.to_csv('2raw_data_4000.csv', encoding = 'utf-8-sig')
files.download('2raw_data_4000.csv')

df_result.to_csv('/content/drive/My Drive/2raw_data_4000.csv', index=False)

Scraping link 0...
Scraping link 1...
Scraping link 2...
Scraping link 3...
Scraping link 4...
Scraping link 5...
Scraping link 6...
Scraping link 7...
Scraping link 8...
Scraping link 9...
Scraping link 10...
Scraping link 11...
Scraping link 12...
Scraping link 13...
Scraping link 14...
Scraping link 15...
Scraping link 16...
Scraping link 17...
Scraping link 18...
Scraping link 19...
Scraping link 20...
Scraping link 21...
Scraping link 22...
Scraping link 23...
Scraping link 24...
Scraping link 25...
Scraping link 26...
Scraping link 27...
Scraping link 28...
Scraping link 29...
Scraping link 30...
Scraping link 31...
Scraping link 32...
Scraping link 33...
Scraping link 34...
Scraping link 35...
Scraping link 36...
Scraping link 37...
Scraping link 38...
Scraping link 39...
Scraping link 40...
Scraping link 41...
Scraping link 42...
Scraping link 43...
Scraping link 44...
Scraping link 45...
Scraping link 46...
Scraping link 47...
Scraping link 48...
Scraping link 49...
Scraping l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>